# İş Birlikçi Filtreleme ( Collaborative Filtering )

Bu konuda karşımıza gelecek 3 temel başlık vardır.

1. Item-Based Collaborative Filtering
2. User-Based Collaborative Filtering
3. Model-Based Collaborative Filtering



### 1. Item-based collaborative filtering (ürün temelli iş birlikçi filtreleme):

In [1]:
# Ürün benzerlikleri üzerinden tavsiyeler yapılır. Buradaki benzerlik içerik benzerliği değildir.
# Örneğin, izlenen bir filmin beğenilme yapısı (puan verilme alışkanlıkları) üzerinden 
# benzer bir beğenilme yapısına sahip başka bir filmi kullanıcıya önerme tekniğidir. 
# Kullanıcıların beğenme alışkanlıkları üzerinden filmler arasında yüksek korelasyon değerlerinden gözlemler yapılır.
# Bu gözlemlere göre en yüksek korelasyonlu filmler önerilir.

In [2]:
# Veri seti: https://grouplens.org/datasets/movielens/

# Adım 1: Veri Setinin Hazırlanması
# Adım 2: User Movie Df'inin Oluşturulması
# Adım 3: Item-Based Film Önerilerinin Yapılması
# Adım 4: Çalışma Scriptinin Hazırlanması

In [3]:
import pandas as pd

Verilerimizi import ediyoruz.

In [4]:
movie = pd.read_csv('datasets/movie_lens_dataset/movie.csv')
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
rating = pd.read_csv('datasets/movie_lens_dataset/rating.csv')
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [6]:
df = movie.merge(rating, how="left", on="movieId")
df.head()

# Verilerimizi merge ediyoruz.

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


### Adım 2: User Movie Df'inin Oluşturulması

In [7]:
# Bu aşamada yaşanan problemlerden bir tanesi de seyreklik durumudur.
# Örneğin bir kullanıcı sadece bir filme puan vermiş olsun; 
# Bir filme puan verdiği halde bu kullanıcı User Movie Df içerisindeki diğer filmlerde de birer hücre teşkil edecektir.
# Bu durum da yapılacak hesaplama işlemlerini geciktirecek ve performans problemlerine sebep olacaktır.
# Dolayısıyla 1000 kere izlenip puan alınan bir filme öneride bulunulabilir, 
# lakin 2 kere izlenip puan almış bir filme ööneride bulunmak çok da doğru bir yaklaşım olmayabilir.

# Bu sebeplerle bir indirgeme işleminde bulunacağız ve 1000'den az puan almış filmleri çalışmanın dışarısında bırakacağız.

df.shape

(20000797, 6)

In [8]:
# Kaç film var bakalım;

df["title"].nunique()

27262

In [9]:
# Hangi filme kaç puan verildiğine bakıyoruz;

df["title"].value_counts().head()

Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: title, dtype: int64

In [10]:
# Yukarıdaki tabloyu bir dataframe'e çevirelim;

comment_counts = pd.DataFrame(df["title"].value_counts())
comment_counts

,title
Pulp Fiction (1994),67310
Forrest Gump (1994),66172
"Shawshank Redemption, The (1994)",63366
"Silence of the Lambs, The (1991)",63299
Jurassic Park (1993),59715
...,...
Rapture (Arrebato) (1980),1
"Education of Mohammad Hussein, The (2013)",1
Satanas (2007),1
Psychosis (2010),1


In [11]:
common_movies_index = comment_counts[comment_counts["title"] > 1000].index
common_movies_index

Index(['Pulp Fiction (1994)', 'Forrest Gump (1994)',
       'Shawshank Redemption, The (1994)', 'Silence of the Lambs, The (1991)',
       'Jurassic Park (1993)', 'Star Wars: Episode IV - A New Hope (1977)',
       'Braveheart (1995)', 'Terminator 2: Judgment Day (1991)',
       'Matrix, The (1999)', 'Schindler's List (1993)',
       ...
       'Pirates of Silicon Valley (1999)',
       'Postman Always Rings Twice, The (1981)', 'Babes in Toyland (1961)',
       'Salton Sea, The (2002)', 'Zulu (1964)', 'Return to Paradise (1998)',
       'Pet Sematary II (1992)', 'Scanners (1981)',
       'Wristcutters: A Love Story (2006)', 'Lincoln Lawyer, The (2011)'],
      dtype='object', length=3159)

In [12]:
common_movies = df[df["title"].isin(common_movies_index)]

common_movies.shape

(17766015, 6)

In [13]:
common_movies.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [14]:
common_movies["title"].nunique()

3159

In [15]:
# Veri setimiz ilk halinde kaç film olduğuna bakacak olursak;

df["title"].nunique()

# 27262 filmden, 3159 filme azalttık.

# Yani 3159 filme 17.7 milyon rate verilirken, 27262 filme 20 milyon rate verilmiş. 
# Bu sebeple yaptığımız indirgeme mantıklı gözüküyor.

27262

In [16]:
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

# Yukarıda pivot table oluşturduk, satırlara userID getirtdik,
# Sütunlara film isimlerini getirdik,
# Kesişimlerine ise rating'i getirdik.

In [17]:
user_movie_df

# Bakıldığında çok fazla bir anlam ifade etmiyor gibi, zira yukarıda bahsedilen seyreklik durumu söz konusudur.
# Biz burada şimdilik bu durum için bir şey yapmayacağız.
# Lakin yine de belirli miktar oy veren kişi sayısına indirgeme işlemi de tercih edilebilir.

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138490.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Adım 3: Item-Based Film Önerilerinin Yapılması

In [18]:
movie_name = "Matrix, The (1999)"
movie_name_column = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name_column).sort_values(ascending=False).head(10)

# Bu yöntem ile önerimizi zenginleştirirken, arka tarafta büyük bir topluluğun fikir birlikteliğiniz almış bulunuyoruz.

title
Matrix, The (1999)                                           1.000000
Matrix Reloaded, The (2003)                                  0.516906
Matrix Revolutions, The (2003)                               0.449588
Animatrix, The (2003)                                        0.367151
Blade (1998)                                                 0.334493
Terminator 2: Judgment Day (1991)                            0.333882
Minority Report (2002)                                       0.332434
Edge of Tomorrow (2014)                                      0.326762
Mission: Impossible (1996)                                   0.320815
Lord of the Rings: The Fellowship of the Ring, The (2001)    0.318726
dtype: float64

In [19]:
movie_name = "Ocean's Twelve (2004)"
movie_name_column = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name_column).sort_values(ascending=False).head(10)

# Bu çalışmanın arka planında bir filme;

    # En benzer 1-5 arasında puan verilme davranışını gösteren diğer filmleri ön plana çıkarma vardır.

title
Ocean's Twelve (2004)                                 1.000000
Ocean's Thirteen (2007)                               0.681654
Ocean's Eleven (2001)                                 0.551280
Eddie (1996)                                          0.474808
National Treasure: Book of Secrets (2007)             0.474230
Eagle Eye (2008)                                      0.473061
Pirates of the Caribbean: On Stranger Tides (2011)    0.472446
Ocean's Eleven (a.k.a. Ocean's 11) (1960)             0.470412
Analyze That (2002)                                   0.459010
Bad Boys II (2003)                                    0.458827
dtype: float64

In [20]:
# Burada tabi filmin bütün ismini girmemiz gerekiyor, bunu bulabilmemiz gerekir.
# Eğer ki bulamazsak bu aramayı da yapamayız.
# Dolayısıyla bizim bir key wor girdiğimizde bu key word'ü içeren filmleri bulabiliyor olmamız gerekir.

def check_film(keyword, user_movie_df):
    return [col for col in user_movie_df.columns if keyword in col]

In [21]:
check_film("Sherlock", user_movie_df)

['Sherlock Holmes (2009)',
 'Sherlock Holmes: A Game of Shadows (2011)',
 'Young Sherlock Holmes (1985)']

In [22]:
movie_name = 'Sherlock Holmes: A Game of Shadows (2011)'
movie_name_column = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name_column).sort_values(ascending=False).head(10)

title
Sherlock Holmes: A Game of Shadows (2011)                                   1.000000
Sherlock Holmes (2009)                                                      0.734823
Old Man and the Sea, The (1958)                                             0.648442
I Shot Andy Warhol (1996)                                                   0.578156
Adventures of Pinocchio, The (1996)                                         0.575200
Beverly Hillbillies, The (1993)                                             0.568271
Boy in the Striped Pajamas, The (Boy in the Striped Pyjamas, The) (2008)    0.539075
Barcelona (1994)                                                            0.533574
Barefoot in the Park (1967)                                                 0.518794
Eddie (1996)                                                                0.504167
dtype: float64